In [ ]:
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from scipy.io import arff 
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import tree
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import collections
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from scipy.io import arff 
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def get_data(list_of_files):
  """creates three lists and appends all the datasets and their Y variable respectively """
  X_ls = []
  Y_ls = []
  Y_label = []
  
  for filename in list_of_files:
    df = pd.read_csv(filename)
    df = df.dropna()

    # getting Y_ls
    Y = df.iloc[:,-1]
    Y = np.array(Y).reshape(-1, 1)
    scaler = MinMaxScaler()
    Y = pd.DataFrame(scaler.fit_transform(Y))
    Y_ls.append(Y)
    
    # getting Y_label using k-means
    yd = DataFrame(Y)
    y = yd.values
    kmeans = KMeans(n_clusters=6).fit(y)
    classified_data = kmeans.labels_
    df_processed = yd.copy()
    df_processed['Cluster Class'] = pd.Series(classified_data, index=df_processed.index)
    y_class = df_processed.iloc[:,-1]
    Y_label.append(y_class)

    # getting X_ls
    X = df.iloc[:, :-1]
    X = X._get_numeric_data()
    X_ls.append(X)

  return(X_ls,Y_ls,Y_label)

In [ ]:
files = glob.glob('/content/drive/My Drive/regression/*.csv')   # create the list of file
X_ls, Y_ls, Y_label = get_data(files)

In [ ]:

def linear_regression(x_var, y_var):
  """Applies Linear regression and returns the r-squared score """
  X_train, X_test, y_train, y_test = train_test_split(x_var, y_var, test_size=0.2, random_state=0)
  lr = LinearRegression()  
  lr.fit(X_train, y_train)
  y_pred = lr.predict(X_test)
  score = metrics.r2_score(y_test, y_pred)
  return(score)


def square(data, feature):
  """convert the feature values to absolute value and then take the square root"""
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = transformed_feature**2
  data_new[feature] = transformed_feature
  return(data_new)
  
def freq(data, feature):
  """convert the feature values to absolute value and then take the frequency"""
  data_new = data.copy()
  given_feature = data_new[feature]
  frequency = collections.Counter(given_feature) 
  transformed_feature = [frequency[x] for x in given_feature] 
  data_new[feature] = transformed_feature
  return(data_new)


def sigmoid(data, feature):
   """convert the feature values to absolute value and then take the sigmoid"""
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = 1/(1 + np.exp(-transformed_feature))
  data_new[feature] = transformed_feature
  return(data_new)

def log(data, feature):
  """convert the feature values to absolute value and then take the log"""
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = preprocessing.MinMaxScaler().fit_transform(transformed_feature)
  transformed_feature = np.log1p(transformed_feature)
  data_new[feature] = transformed_feature
  return(data_new)

def average(array):
  """Takes the average"""
  avg = sum(array)/len(array)
  return(avg)

def scale(data):
  """Scales the original data"""
  scaled_x = preprocessing.MinMaxScaler().fit_transform(data)
  data = pd.DataFrame(data = scaled_x, columns = data.columns)
  scaled_data = data.copy()
  return(scaled_data)


def square_root(data, feature):
  """convert the feature values to absolute value and then take the square root"""
  data_new = data.copy()
  transformed_feature = data_new[feature]
  transformed_feature = abs(transformed_feature)
  transformed_feature = np.sqrt(transformed_feature)
  data_new[feature] = transformed_feature
  return(data_new)

def min_max_scale(data):
  """Scales the data using min-max scaler"""
    scaler = MinMaxScaler()
    scaler = scaler.fit(data)
    scaled_value = scaler.transform(data)
    return(scaled_value)



In [ ]:
#Automating the feature transformations

def feature_trans(y, data, features_list, perc,transformation):
  """Applies the same transformation for each feature and evaluates the model before and after,
  returns a classified training data of positive and negatiive training samples"""
  
  pos_samples= []
  neg_samples = []
  
  
  original_score = linear_regression(data, y)
  
  threshold = perc

  #threshold = round(((1-original_score)*perc), 3) #setting the threshold as a percentage of the original score
  
  # Applying Transformation
  for col in features_list:
    if transformation == 'sqr':  
      x = square_root(data, col)
    elif transformation == 'square':
      x = square(data, col)
    elif transformation == 'freq':
      x = freq(data, col)
    elif transformation == 'log':
      x = log(data,col)
    elif transformation == 'sig':
      x = sigmoid(data, col)
    else:
      raise RuntimeError('No specified transformation!')

    
    new_score = linear_regression(x, y)
    
    if round(new_score, 3) - round(original_score, 3) >= threshold:
      pos_samples.append(col)
    else:
      neg_samples.append(col)
 # creating positive and negative samples        
  positive_samples = [y for x in pos_samples for y in x]
  negative_samples = [y for x in neg_samples for y in x]
  
  return(positive_samples, negative_samples)

In [ ]:
def binning_norm(data, b):
    """takes in a list of values (data) and number of bins b and bins the values of the feature according 
  to the bin edges produced by the KBinsDiscretizer function, and returns the normalized value for each bin"""
  model = KBinsDiscretizer(n_bins=b, encode='ordinal', strategy='uniform')

  model.fit(data.reshape(-1,1))

  bin_edges = model.bin_edges_[0]
  bin_values = []
  bin_edges = np.delete(bin_edges, 0)
  old_ran = 0
   #counting number of values in each bin
  for x,ran in enumerate(bin_edges):
    if x == len(bin_edges)-1:
      num = sum(1 for x in data if x <= ran and x >= old_ran)
    else:
      num = sum(1 for x in data if x < ran and x >= old_ran)
    old_ran = ran
    bin_values.append(num)
  bin_values = [y/len(data) for y in bin_values]
  return(bin_values)
    
#automate QSA
#using normalized
def binning_normalized(features_list, data, y, class_, bins):
   """given a list of features, divides the values into two lists accrording to the class,
  then scales the values and applies binning norm function, then returns the fixed-size arrays of the data"""
    
  bin_edge_pos = []
  bin_edge_neg = []
  b = {"bin_edges":[], "class": []}
  
  for feature in features_list:

    data_feature = list(data[feature])
    x_f_0 = [z for x,z in enumerate(data_feature) if y[x] == 0]
    x_f_1 = [z for x,z in enumerate(data_feature) if y[x] == 1]
    x_f_2 = [z for x,z in enumerate(data_feature) if y[x] == 2]
    x_f_3 = [z for x,z in enumerate(data_feature) if y[x] == 3]
    x_f_4 = [z for x,z in enumerate(data_feature) if y[x] == 4]
    x_f_5 = [z for x,z in enumerate(data_feature) if y[x] == 5]
    
    # reshaping the data
    x_f_0 = np.array(x_f_0).reshape(-1,1)
    x_f_1 = np.array(x_f_1).reshape(-1,1)
    x_f_2 = np.array(x_f_2).reshape(-1,1)
    x_f_3 = np.array(x_f_3).reshape(-1,1)
    x_f_4 = np.array(x_f_4).reshape(-1,1)
    x_f_5 = np.array(x_f_5).reshape(-1,1)
    
    


    # scaling the data
    x_f_0 = min_max_scale(x_f_0)
    x_f_1 = min_max_scale(x_f_1)
    x_f_2 = min_max_scale(x_f_2)
    x_f_3 = min_max_scale(x_f_3)
    x_f_4 = min_max_scale(x_f_4)
    x_f_5 = min_max_scale(x_f_5)
 


    # creating training data and labels

    bin_all = []
    bin_all.extend(binning_norm(x_f_0, bins))
    bin_all.extend(binning_norm(x_f_1, bins))
    bin_all.extend(binning_norm(x_f_2, bins))
    bin_all.extend(binning_norm(x_f_3, bins))
    bin_all.extend(binning_norm(x_f_4, bins))
    bin_all.extend(binning_norm(x_f_5, bins))
  

    b["bin_edges"].append(bin_all) 
    b["class"].append(class_)
      
  training_data = np.array(b["bin_edges"])
  labels = np.array(b["class"])

  return(training_data, labels)

In [ ]:

def make_seq(inc):
   """creates a sequence of numbers between 0 and 100, and increments it by the specified inc"""
  x = []
  start = 0
  end = 100

  while start < end:
    start += inc 
    start = round(start,2)
    x.append(start)
  return(x)

# Automate Percentile Binning
def binning_quantile(features_list, data, y, class_, bins):
  """creates a feature representation of the features in the feature list using
    quantiles by binning all the values lower than a given quantile into one bin, returns
    a list of the fixed-size arrays of features along with their label as positive or negative samples"""
    
  b = {"bin_edges":[], "class": []}
  quantiles = make_seq(1)

  for feature in features_list:

    data_feature = list(data[feature])
    x_f_0 = [z for x,z in enumerate(data_feature) if y[x] == 0]
    x_f_1 = [z for x,z in enumerate(data_feature) if y[x] == 1]
    x_f_2 = [z for x,z in enumerate(data_feature) if y[x] == 2]
    x_f_3 = [z for x,z in enumerate(data_feature) if y[x] == 3]
    x_f_4 = [z for x,z in enumerate(data_feature) if y[x] == 4]
    x_f_5 = [z for x,z in enumerate(data_feature) if y[x] == 5]

# reshaping the data
    x_f_0 = np.array(x_f_0).reshape(-1,1)
    x_f_1 = np.array(x_f_1).reshape(-1,1)
    x_f_2 = np.array(x_f_2).reshape(-1,1)
    x_f_3 = np.array(x_f_3).reshape(-1,1)
    x_f_4 = np.array(x_f_4).reshape(-1,1)
    x_f_5 = np.array(x_f_5).reshape(-1,1)


    from sklearn.preprocessing import MinMaxScaler
# scaling the data
    x_f_0 = min_max_scale(x_f_0)
    x_f_1 = min_max_scale(x_f_1)
    x_f_2 = min_max_scale(x_f_2)
    x_f_3 = min_max_scale(x_f_3)
    x_f_4 = min_max_scale(x_f_4)
    x_f_5 = min_max_scale(x_f_5)
   

    bin_0 = []
    bin_1 = []
    bin_2 = []
    bin_3 = []
    bin_4 = []
    bin_5 = []


    for quant in quantiles:
      bin_0.append(np.percentile(x_f_0, quant))
      bin_1.append(np.percentile(x_f_1, quant))
      bin_2.append(np.percentile(x_f_2, quant))
      bin_3.append(np.percentile(x_f_3, quant))
      bin_4.append(np.percentile(x_f_4, quant))
      bin_5.append(np.percentile(x_f_5, quant)) 
           
 # creating training data and labels     
    bin_all = []
    bin_all.extend(bin_0)
    bin_all.extend(bin_1)
    bin_all.extend(bin_2)
    bin_all.extend(bin_3)
    bin_all.extend(bin_4)
    bin_all.extend(bin_5)

    b["bin_edges"].append(bin_all) 
    b["class"].append(class_)
      
  training_data = np.array(b["bin_edges"])
  labels = np.array(b["class"])

  return(training_data, labels)

In [ ]:
# Discarding datasets with r-squared score greater than 0.97

new_x_ls, new_y_ls, new_y_class = [], [], []

for x_var, y_var, y_class in zip(X_ls, Y_ls, Y_label):
  
  scaled_dataset = scale(x_var)
  score = linear_regression(x_var, y_var)
  if score <= 0.97:
    new_x_ls.append(x_var)

    new_y_ls.append(y_var)
    
    new_y_class.append(y_class)
  

In [ ]:
import pickle
with open('new_x_ls_linr.pickle', 'wb') as handle:
    pickle.dump(new_x_ls, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('new_y_ls_linr.pickle', 'wb') as handle:
    pickle.dump(new_y_ls, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('new_y_class_linr.pickle', 'wb') as handle:
    pickle.dump(new_y_class, handle, protocol=pickle.HIGHEST_PROTOCOL)

from google.colab import files
files.download('new_x_ls_linr.pickle') 
files.download('new_y_ls_linr.pickle')
files.download('new_y_class_linr.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# applying log trasformation on positive and negatice samples
i = 1
pos_list = []
neg_list = []

for dataset, label in zip(new_x_ls, new_y_ls):
  print("Training dataset " + str(i) + "...")

  scaled_dataset = scale(dataset)

  features = [[col] for col in scaled_dataset.columns]

  

  positive_samples, negative_samples = feature_trans(label, scaled_dataset, features, 0.01,"log")

  pos_list.append(positive_samples)
  neg_list.append(negative_samples)

  i +=1 

Training dataset 1...
Training dataset 2...
Training dataset 3...
Training dataset 4...
Training dataset 5...
Training dataset 6...
Training dataset 7...
Training dataset 8...
Training dataset 9...
Training dataset 10...
Training dataset 11...
Training dataset 12...
Training dataset 13...
Training dataset 14...
Training dataset 15...
Training dataset 16...
Training dataset 17...
Training dataset 18...
Training dataset 19...
Training dataset 20...
Training dataset 21...
Training dataset 22...
Training dataset 23...
Training dataset 24...
Training dataset 25...
Training dataset 26...
Training dataset 27...
Training dataset 28...
Training dataset 29...
Training dataset 30...
Training dataset 31...
Training dataset 32...
Training dataset 33...
Training dataset 34...
Training dataset 35...
Training dataset 36...
Training dataset 37...
Training dataset 38...
Training dataset 39...
Training dataset 40...
Training dataset 41...
Training dataset 42...
Training dataset 43...
Training dataset 44.

In [ ]:
# creating training data for MLPs
training_data = []
training_labels = []
i = 1
for i, (dataset, label) in enumerate(zip(new_x_ls,new_y_class)):
  
  print("Training dataset " + str(i) + "...")
  if len(pos_list[i]) == 0:
      neg_training, neg_labels = binning_quantile(neg_list[i], dataset, label, 0,100)
      training_data.extend(neg_training)
      training_labels.extend(neg_labels)
  elif len(neg_list[i]) == 0:
    pos_training, pos_labels = binning_quantile(pos_list[i], dataset, label, 1, 100)
    training_data.extend(pos_training)
    training_labels.extend(pos_labels)
  else:
    pos_training, pos_labels = binning_quantile(pos_list[i], dataset, label, 1, 100)
    neg_training, neg_labels = binning_quantile(neg_list[i], dataset, label, 0, 100)
    training_data.extend(pos_training)
    training_data.extend(neg_training) 
    training_labels.extend(pos_labels)
    training_labels.extend(neg_labels) 
  i+=1

Training dataset 0...
Training dataset 1...
Training dataset 2...
Training dataset 3...
Training dataset 4...
Training dataset 5...
Training dataset 6...
Training dataset 7...
Training dataset 8...
Training dataset 9...
Training dataset 10...
Training dataset 11...
Training dataset 12...
Training dataset 13...
Training dataset 14...
Training dataset 15...
Training dataset 16...
Training dataset 17...
Training dataset 18...
Training dataset 19...
Training dataset 20...
Training dataset 21...
Training dataset 22...
Training dataset 23...
Training dataset 24...
Training dataset 25...
Training dataset 26...
Training dataset 27...
Training dataset 28...
Training dataset 29...
Training dataset 30...
Training dataset 31...
Training dataset 32...
Training dataset 33...
Training dataset 34...
Training dataset 35...
Training dataset 36...
Training dataset 37...
Training dataset 38...
Training dataset 39...
Training dataset 40...
Training dataset 41...
Training dataset 42...
Training dataset 43..

In [ ]:
import pickle
with open('train_linears_log01.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('label_linears_log01.pickle', 'wb') as handle:
    pickle.dump(training_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

# from google.colab import files
# files.download('train_dec_log0.pickle') 
# files.download('label_dec_log75-2.pickle') 